## Keras 函数式API

In [1]:
from keras import Input, layers

# 一个张量
input_tensor = Input(shape = (32,))

# 一个层是一个函数
dense = layers.Dense(32, activation = 'relu')

# 可以在一个张量上调用一个层,它会返回一个张量
output_tensor = dense(input_tensor)

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# Sequential模型
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation = 'relu', input_shape = (64,)))
seq_model.add(layers.Dense(32, activation = 'relu'))
seq_model.add(layers.Dense(10, activation = 'softmax'))

# 对应的函数式API实现
input_tensor = Input(shape = (64,))
x = layers.Dense(32, activation = 'relu')(input_tensor)
x = layers.Dense(32, activation = 'relu')(x)
output_tensor = layers.Dense(10, activation = 'softmax')(x)
model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer = 'rmsprop',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000,10))
model.fit(x_train, y_train, epochs = 10, batch_size = 128)
scores = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 1s 702us/step - loss: 11.8067 - acc: 0.0960
Epoch 2/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.6078 - acc: 0.0920
Epoch 3/10
1000/1000 [==============================] - 0s 27us/step - loss: 11.5825 - acc: 0.0980
Epoch 4/10
1000/1000 [==============================] - 0s 32us/step - loss: 11.5715 - acc: 0.1010
Epoch 5/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.5638 - acc: 0.1060
Epoch 6/10
1000/1000 [==============================] - 0s 28us/step - loss: 11.5575 - acc: 0.1190
Epoch 7/10
1000/1000 [==============================] - 0s 33us/step - loss: 11.5532 - acc: 0.1160
Epoch 8/10
1000/1000 [==============================] - 0s 29us/step - loss: 11.5498 - acc: 0.1130
Epoch 9/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.5467 - acc: 0.1170
Epoch 10/10
1000/1000 [==============================] - 0s 46us/step


In [6]:
print(scores)

[11.540735137939453, 0.123]


In [7]:
model.metrics_names

['loss', 'acc']

In [22]:
type(scores)


list

## 用函数式API实现双输入问答模型

In [23]:
from keras.models import Model
from keras import layers
from keras import Input 

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape = (None, ), dtype = 'int32', name = 'text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape = (None,), dtype = 'int32', name = 'question')
embedded_question= layers.Embedding(question_vocabulary_size, 64)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis = 1)

answer = layers.Dense(answer_vocabulary_size, activation = 'softmax')(concatenated)


In [24]:
model = Model([text_input, question_input], answer)
model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

In [27]:
import keras

In [29]:
num_samples = 1000
max_length = 100
text = np.random.randint(1, text_vocabulary_size, size = (num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size = (num_samples, max_length))
answer = np.random.randint(answer_vocabulary_size, size = (num_samples))

answer = keras.utils.to_categorical(answer, answer_vocabulary_size)

model.fit([text, question], answer, epochs = 10, batch_size = 64)

model.fit({'text':text, 'question':question}, answer, epochs = 10, batch_size = 64)

Epoch 1/10
1000/1000 [==============================] - 4s 4ms/step - loss: 6.2148 - acc: 1.0000e-03
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1896 - acc: 0.0270
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.0834 - acc: 0.0050
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.9746 - acc: 0.0070
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.8662 - acc: 0.0080
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.7504 - acc: 0.0060
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6190 - acc: 0.0140
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.5034 - acc: 0.0170
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.3924 - acc: 0.0410
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.2948 - acc: 0.0500
Epoch 1/10
1000/1000 [===